In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
train_fp = '../input/tabular-playground-series-feb-2022/train.csv'

train_df = pd.read_csv(train_fp)

test_fp = '../input/tabular-playground-series-feb-2022/test.csv'

test_df = pd.read_csv(test_fp)

In [ ]:
train_df

In [ ]:
train_df.info()
print(" ")
train_df.isnull().sum()

In [ ]:
test_df

In [ ]:
test_df.info()
print(" ")
test_df.isnull().sum()

In [ ]:
train_copy = train_df.copy()
test_copy = test_df.copy()

In [ ]:
train = train_copy.drop(['row_id'], axis=1)
test = test_copy.drop(['row_id'], axis=1)


X = train.drop(['target'], axis=1)
y = train_copy.target

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y_le = encoder.fit_transform(y)

In [ ]:
X

In [ ]:
y_le

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

xtrain,xvalid,ytrain,yvalid = train_test_split(X,y_le,test_size=0.2,random_state=0)

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()

model = xgb.fit(xtrain, ytrain)

preds = model.predict(xvalid)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

cv = KFold(n_splits=5, random_state=1, shuffle=True)

scores = -1 * cross_val_score(model, X, y_le, cv=cv, 
                              scoring='neg_mean_absolute_error', 
                              n_jobs=1)
print('MAE Scores: ', scores)
print('Average score: ', scores.mean())

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score


print("Mean absolute error: {}".format(mean_absolute_error(yvalid[:100000], preds)))
print("r2 score: {}".format(r2_score(yvalid[:100000], preds)))

In [ ]:
sample = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

In [ ]:
final_preds = model.predict(test)

In [ ]:
submission = sample.copy()

output = encoder.inverse_transform(final_preds)

submission['target'] = output

submission

In [ ]:
submission.to_csv('submission.csv', index=False)